In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import random
import pickle
import datetime
import os
import torch.nn.functional as F
from scipy.stats import special_ortho_group
from scipy.spatial.transform import Rotation as R
import torch.nn.functional as F
from pytorch_metric_learning import miners, losses
import wandb

In [ ]:
seed=999
os.environ["PL_GLOBAL_SEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
def use_GPU():
    """ This function activates the gpu 
    """
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(torch.cuda.get_device_name(0), "is available and being used")
    else:
        device = torch.device("cpu")
        print("GPU is not available, using CPU instead") 
    return device  


def center_in_origin(frag):
    """ This function normalize each fragment in (0,1)
    """
    min_vals, _ = torch.min(frag[:, 0:3], axis=0)
    max_vals, _ = torch.max(frag[:, 0:3], axis=0)
    frag[:, 0:3] = (frag[:, 0:3] - min_vals) / (max_vals - min_vals)
    
    return frag
    
def normalize(batch):
    """ This function apply center_in_origin() to each fragment in the batch
    """
    out=[]
    for element in batch:
        out.append(center_in_origin(element))
    out_tensor = torch.stack(out)    
    return out_tensor  



def translate_to_origin(frag):
    """ This function translate each fragment in the origin
    """
    frag[:,:3] -= torch.mean(frag[:,:3]) 
    return frag

def apply_translation(batch):
    """ This function apply translate_to_origin() to each fragment in the batch
    """
    out=[]
    for element in batch:
        out.append(translate_to_origin(element))
    out_tensor = torch.stack(out)    
    return out_tensor


def random_rotation(frag):

    randrot = (torch.rand(3)*360).tolist()
    r = R.from_euler('zyx', randrot, degrees=True)
    frag[:,:3] = torch.from_numpy(r.apply(frag[:,:3]))
    frag[:,3:6] = torch.from_numpy(r.apply(frag[:,3:6]))
    return frag

def apply_randomrotations(batch):
    """ This function apply random_rotation() to each fragment in the batch
    """
    out=[]
    for element in batch:
        out.append(random_rotation(element))
    out_tensor = torch.stack(out)    
    return out_tensor

In [ ]:
# Code form the repository of PCT https://github.com/qq456cvb/Point-Transformers

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    return torch.sum((src[:, :, None] - dst[:, None]) ** 2, dim=-1)

def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S, [K]]
    Return:
        new_points:, indexed points data, [B, S, [K], C]
    """
    raw_size = idx.size()
    idx = idx.reshape(raw_size[0], -1)
    res = torch.gather(points, 1, idx[..., None].expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_size, -1)


def farthest_point_sample(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        distance = torch.min(distance, dist)
        farthest = torch.max(distance, -1)[1]
    return centroids

def sample_and_group(npoint, nsample, xyz, points):
    B, N, C = xyz.shape
    S = npoint 
    
    fps_idx = farthest_point_sample(xyz, npoint) # [B, npoint]

    new_xyz = index_points(xyz, fps_idx) 
    new_points = index_points(points, fps_idx)

    dists = square_distance(new_xyz, xyz)  # B x npoint x N
    idx = dists.argsort()[:, :, :nsample]  # B x npoint x K

    grouped_points = index_points(points, idx)
    grouped_points_norm = grouped_points - new_points.view(B, S, 1, -1)
    new_points = torch.cat([grouped_points_norm, new_points.view(B, S, 1, -1).repeat(1, 1, nsample, 1)], dim=-1)
    return new_xyz, new_points


class Local_op(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        b, n, s, d = x.size()  # torch.Size([32, 512, 32, 6]) 
        x = x.permute(0, 1, 3, 2)
        x = x.reshape(-1, d, s)
        batch_size, _, N = x.size()
        x = self.relu(self.bn1(self.conv1(x))) # B, D, N
        x = self.relu(self.bn2(self.conv2(x))) # B, D, N
        x = torch.max(x, 2)[0]
        x = x.view(batch_size, -1)
        x = x.reshape(b, n, -1).permute(0, 2, 1)
        return x


class SA_Layer(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.q_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.k_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.q_conv.weight = self.k_conv.weight 
        self.v_conv = nn.Conv1d(channels, channels, 1)
        self.trans_conv = nn.Conv1d(channels, channels, 1)
        self.after_norm = nn.BatchNorm1d(channels)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x_q = self.q_conv(x).permute(0, 2, 1) # b, n, c 
        x_k = self.k_conv(x)# b, c, n        
        x_v = self.v_conv(x)
        energy = x_q @ x_k # b, n, n 
        attention = self.softmax(energy)
        attention = attention / (1e-9 + attention.sum(dim=1, keepdims=True))
        x_r = x_v @ attention # b, c, n 
        x_r = self.act(self.after_norm(self.trans_conv(x - x_r)))
        x = x + x_r
        return x
    



    

class StackedAttention(nn.Module):
    def __init__(self, channels=256):
        super().__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)

        self.bn1 = nn.BatchNorm1d(channels)
        self.bn2 = nn.BatchNorm1d(channels)

        self.sa1 = SA_Layer(channels)
        self.sa2 = SA_Layer(channels)
        self.sa3 = SA_Layer(channels)
        self.sa4 = SA_Layer(channels)

        self.relu = nn.ReLU()
        
    def forward(self, x):
        # 
        # b, 3, npoint, nsample  
        # conv2d 3 -> 128 channels 1, 1
        # b * npoint, c, nsample 
        # permute reshape
        batch_size, _, N = x.size()

        x = self.relu(self.bn1(self.conv1(x))) # B, D, N
        x = self.relu(self.bn2(self.conv2(x)))

        x1 = self.sa1(x)
        x2 = self.sa2(x1)
        x3 = self.sa3(x2)
        x4 = self.sa4(x3)
        
        x = torch.cat((x1, x2, x3, x4), dim=1)

        return x

In [ ]:
class SA_Layer_classic(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.q_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.k_conv = nn.Conv1d(channels, channels // 4, 1, bias=False)
        self.q_conv.weight = self.k_conv.weight 
        self.v_conv = nn.Conv1d(channels, channels, 1)
        self.trans_conv = nn.Conv1d(channels, channels, 1)
        self.after_norm = nn.BatchNorm1d(channels)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x_q = self.q_conv(x).permute(0, 2, 1) # b, n, c 
        x_k = self.k_conv(x)# b, c, n        
        x_v = self.v_conv(x)
        energy = x_q @ x_k # b, n, n 
        attention = self.softmax(energy)
        attention = attention / (1e-9 + attention.sum(dim=1, keepdims=True))
        x_r = x_v @ attention # b, c, n 
        x_r = self.act(self.after_norm(self.trans_conv(x_r)))
        
        return x_r


class StackedAttention_classic(nn.Module):
    def __init__(self, channels=256):
        super().__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size=1, bias=False)

        self.bn1 = nn.BatchNorm1d(channels)
        self.bn2 = nn.BatchNorm1d(channels)

        self.sa1 = SA_Layer_classic(channels)
        self.sa2 = SA_Layer_classic(channels)
        self.sa3 = SA_Layer_classic(channels)
        self.sa4 = SA_Layer_classic(channels)

        self.relu = nn.ReLU()
        
    def forward(self, x):
        # 
        # b, 3, npoint, nsample  
        # conv2d 3 -> 128 channels 1, 1
        # b * npoint, c, nsample 
        # permute reshape
        batch_size, _, N = x.size()

        x = self.relu(self.bn1(self.conv1(x))) # B, D, N
        x = self.relu(self.bn2(self.conv2(x)))

        x1 = self.sa1(x)
        x2 = self.sa2(x1)
        x3 = self.sa3(x2)
        x4 = self.sa4(x3)
        
        x = torch.cat((x1, x2, x3, x4), dim=1)

        return x





class Branch(nn.Module):
    def __init__(self):
        super().__init__()
        
        d_points = 7 # we have 7 features for each point
        self.conv1 = nn.Conv1d(d_points, 64, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.gather_local_0 = Local_op(in_channels=128, out_channels=128)
        self.gather_local_1 = Local_op(in_channels=256, out_channels=256)
        self.pt_last = StackedAttention()###############################################################

        self.relu = nn.ReLU()
        self.conv_fuse = nn.Sequential(nn.Conv1d(1280, 1024, kernel_size=1, bias=False),
                                   nn.BatchNorm1d(1024),
                                   nn.LeakyReLU(negative_slope=0.2))

        
    def forward(self, x):
        xyz = x[..., :3]
        x = x.permute(0, 2, 1)
        batch_size, _, _ = x.size()
        x= x.double()
        x = self.relu(self.bn1(self.conv1(x))) # B, D, N
        x = self.relu(self.bn2(self.conv2(x))) # B, D, N
        x = x.permute(0, 2, 1)
        new_xyz, new_feature = sample_and_group(npoint=512, nsample=32, xyz=xyz, points=x)         
        feature_0 = self.gather_local_0(new_feature)
        feature = feature_0.permute(0, 2, 1)
        new_xyz, new_feature = sample_and_group(npoint=256, nsample=32, xyz=new_xyz, points=feature) 
        feature_1 = self.gather_local_1(new_feature)
        
        x = self.pt_last(feature_1)
        x = torch.cat([x, feature_1], dim=1)
        x = self.conv_fuse(x)
        x = torch.max(x, 2)[0] # Returns the maximum value of all elements in the input tensor. (2 elementes for each vector)
        x = x.view(batch_size, -1) # Returns a new tensor with the same data as the self tensor but of a different shape.
        
        return x

In [ ]:
# data loading
train = torch.load("C:\\Users\\Alessandro\\Desktop\\Tesi\\pair_dataset\\dataset_1024_AB\\train_pair_dataset_REG.pt")
val = torch.load("C:\\Users\\Alessandro\\Desktop\\Tesi\\pair_dataset\\dataset_1024_AB\\val_pair_dataset_REG.pt")
test = torch.load("C:\\Users\\Alessandro\\Desktop\\Tesi\\pair_dataset\\dataset_1024_AB\\test_pair_dataset_REG.pt")

In [ ]:
# let's find the largest clusters

count = 0
indices = []
count_val = 0
indices_val = []

for i in range(0, 1526):
    if train[i][0].shape[0] > 70:
        count += 1
        indices.append(i)

for i in range(0, val.shape[0]):
    if val[i][0].shape[0] > 70:
        count_val += 1
        indices_val.append(i)

print("Positions to remove (Train):", indices)
print("Positions to remove (Val):", indices_val)

Positions to remove (Train): [2, 14, 20, 22, 28, 30, 31, 35, 36, 39, 71, 78, 87, 91, 95, 109, 136, 148, 163, 167, 192, 197, 205, 209, 215, 217, 218, 222, 228, 241, 255, 261, 263, 265, 273, 284, 300, 308, 318, 337, 344, 346, 353, 361, 364, 369, 398, 420, 423, 427, 431, 433, 468, 469, 471, 474, 489, 494, 496, 507, 513, 516, 517, 538, 547, 551, 564, 577, 591, 595, 597, 600, 607, 613, 617, 633, 635, 644, 651, 652, 660, 664, 675, 689, 694, 699, 700, 707, 711, 721, 731, 735, 754, 757, 758, 763, 768, 771, 776, 783, 789, 813, 818, 831, 834, 837, 841, 844, 846, 874, 879, 896, 909, 917, 931, 943, 971, 977, 979, 988, 991, 994, 996, 999, 1042, 1045, 1050, 1057, 1077, 1078, 1079, 1084, 1100, 1109, 1118, 1126, 1127, 1129, 1130, 1131, 1143, 1150, 1156, 1188, 1217, 1219, 1227, 1254, 1274, 1284, 1290, 1295, 1299, 1316, 1324, 1334, 1336, 1337, 1347, 1353, 1356, 1390, 1392, 1412, 1418, 1460, 1463, 1466, 1469, 1474, 1483, 1489, 1498, 1499, 1504, 1505, 1507, 1514]
Positions to remove (Val): [6, 12, 13, 22,

In [ ]:
# We are removing the largest clusters for computational reasons
mask = torch.ones(train.shape[0], dtype=torch.bool)
mask[indices] = False
filtered_tensor = train[mask]
train = filtered_tensor

mask_val = torch.ones(val.shape[0], dtype=torch.bool)
mask_val[indices_val] = False
filtered_tensor_val = val[mask_val]
val = filtered_tensor_val

In [ ]:
count_test = 0
indices_test = []

for i in range(0, 327):
    if test[i][0].shape[0] > 70:
        count_test += 1
        indices_test.append(i)
mask_test = torch.ones(test.shape[0], dtype=torch.bool)
mask_test[indices_test] = False
filtered_tensor_test = test[mask_test]
test= filtered_tensor_test     

In [ ]:
print(train.shape)
print(val.shape)
print(test.shape)

(1348, 2)
(292, 2)
(289, 2)


In [ ]:
def adjmatrix_into_y(adj_mat):

    """ This function maps the adj_matrix into a vector (leverage the matrix symmetry to save computations)
    """
    labels = []
    
    # Discover the number of fragments
    n_frags = len(adj_mat[0])
    
    for j in range(0, n_frags - 1):
        init = j + 1
        for k in range(init, n_frags):
            
            labels.append(adj_mat[j][k])
    
    return  labels

def max_the1_v (vector):

    """  This function outputs a vector containing the positions of fragment pairs to extract in order to ensure generating the maximum possible number of balanced pairs.
    """

    how_many_ones = 0
    how_many_zeroes = 0

    positions_0 = []
    positions_1 = []
 

    for i, elemento in enumerate(vector):
        if elemento == 1 :
            how_many_ones += 1
            positions_1.append(i)


        elif elemento == 0 :
            how_many_zeroes += 1 
            positions_0.append(i)
    
    # find the min = k 
    selected_value=min(how_many_ones, how_many_zeroes)

    #  select only the first k elements from each vector
    positions_1_sample = positions_1[:selected_value]
    positions_0_sample = positions_0[:selected_value]

    posizioni =   positions_1_sample + positions_0_sample
    posizioni = sorted(posizioni)  
    return posizioni

def all_pos (vector):

    """  This function outputs a vector containing the positions of fragment pairs to extract in order to ensure generating the maximum possible number of balanced pairs.
    """

   
    positions = []

    for i, elemento in enumerate(vector):
        
        positions.append(i)
    posizioni = sorted(positions)  
    return posizioni


def max_the1_mod (vector):

    """  This function outputs a vector containing the positions of fragment pairs to extract in order to ensure generating the maximum possible number of balanced pairs.
    """

    how_many_ones = 0
    how_many_zeroes = 0

    positions_0 = []
    positions_1 = []
 

    for i, elemento in enumerate(vector):
        if elemento == 1 :
            how_many_ones += 1
            positions_1.append(i)


        elif elemento == 0 :
            how_many_zeroes += 1 
            positions_0.append(i)
    
    # find the min = k 
    selected_value=min(how_many_ones, how_many_zeroes)

    if how_many_ones > how_many_zeroes:
         extra_1= int((how_many_ones-how_many_zeroes)/3)
         extra_0= 0
    elif how_many_ones < how_many_zeroes:
         extra_1= 0
         extra_0= int((how_many_zeroes-how_many_ones)/3)
    else:
        extra_1= 0
        extra_0= 0


    #  select only the first k elements from each vector
    positions_1_sample = positions_1[:selected_value + extra_1]
    positions_0_sample = positions_0[:selected_value + extra_0]

    posizioni =   positions_1_sample + positions_0_sample
    posizioni = sorted(posizioni)  
    return posizioni  



def max_the_max (vector):

    """  This function outputs a vector containing the positions of fragment pairs to extract in order to ensure generating the maximum possible number of balanced pairs.
    """

    how_many_ones = 0
    how_many_zeroes = 0

    positions_0 = []
    positions_1 = []
 

    for i, elemento in enumerate(vector):
        if elemento == 1 :
            how_many_ones += 1
            positions_1.append(i)


        elif elemento == 0 :
            how_many_zeroes += 1 
            positions_0.append(i)
    
    # find the min = k 
    selected_value=min(how_many_ones, how_many_zeroes)

    if how_many_ones > how_many_zeroes:

         positions_1_sample = random.sample(positions_1, selected_value)

         positions_0_sample = positions_0[:selected_value]

    elif how_many_ones < how_many_zeroes:

         positions_1_sample = positions_1[:selected_value] 
         positions_0_sample = random.sample(positions_0, selected_value)

    else:
        positions_1_sample = positions_1[:selected_value]
        positions_0_sample = positions_0[:selected_value]


    posizioni =   positions_1_sample + positions_0_sample
    posizioni = sorted(posizioni)  
    return posizioni



def max_the1_samples (vector):

    how_many_ones = 0
    how_many_zeroes = 0


    positions_0 = []
    positions_1 = []
 

    for i, elemento in enumerate(vector):
        if elemento == 1 :
            how_many_ones += 1
            positions_1.append(i)


        elif elemento == 0 :
            how_many_zeroes += 1 
            positions_0.append(i)

    selected_value=min(how_many_ones, how_many_zeroes)
  
    positions_1_sample = random.sample(positions_1, int((selected_value*3)/4))
    positions_0_sample = random.sample(positions_0, int((selected_value*3)/4))

    posizioni =   positions_1_sample + positions_0_sample
    posizioni = sorted(posizioni)  
    return posizioni




    
def CreateCouples_intra_train_masked_pos(cluster_of_pt, adj_mat, posizioni):

    """ This function is used to create the couples from the cluster in which each element is already preprocessed from the pct. 
        It doesn't create all possible pairs; in fact, by using the position vector, we know in which positions the pairs we need to extract are located
        (those that allow us to have the maximum number of balanced pairs within the cluster).
    
    Input:
        cluster_of_pt: the cluster of pointcloud (alredy processed from the PCT)
        adj_mat: the adjacency matrix associated to the cluster
        posizioni: the vector of positions obatined from max_the1_v() function
    Return:
        frag_a: the tensor of the first element of each couple
        frag_b: the tensor of the second element of each couple
        labels: a binary value that indicates if the couple is adjacent (1) or not (0)
    """
    
    frag_a = []
    frag_b = []
    labels = []
    pos_realtime=0
    # Discover the number of fragments
    n_frags = len(adj_mat[0])
    
    for j in range(0, n_frags - 1):
        init = j + 1
        

        for k in range(init, n_frags):
            if pos_realtime in posizioni:  # Tests whether pos_realtime is contained in positions
                frag_a_tensor = cluster_of_pt[j]
                frag_b_tensor = cluster_of_pt[k]
                label_value = adj_mat[j][k]

                # Stack of tensors and labels at each step
                if len(frag_a) == 0:
                    frag_a = frag_a_tensor.unsqueeze(0)
                    frag_b = frag_b_tensor.unsqueeze(0)
                    labels = torch.tensor([label_value])
                else:
                    frag_a = torch.cat((frag_a, frag_a_tensor.unsqueeze(0)), dim=0)
                    frag_b = torch.cat((frag_b, frag_b_tensor.unsqueeze(0)), dim=0)
                    labels = torch.cat((labels, torch.tensor([label_value])), dim=0)

            pos_realtime = pos_realtime + 1
    
    return frag_a, frag_b, labels




In [ ]:
class Model2_mod(nn.Module):
    def __init__(self):
        super().__init__()

        self.ptc_net = Branch()
        self.pair_net = PairModel2()      

    def forward(self, matrix, cluster, type_):
        matrix = matrix.numpy()
        # let's convert the matrix into a vector
        y = adjmatrix_into_y(matrix[0])
        # select the same number of 0 and 1 in the previous vector

        if type_ == 0:
            # all the position (unbalanced)
            positions = all_pos(y)
        elif type_ == 1:
            # same 0 and 1 (balanced)
            positions = max_the1_v(y)
        elif type_ == 2:
            # not the same 0 and 1 (little unbalanced)
            positions = max_the1_mod(y)
        elif type_ == 3: 
            # same 0 and 1 + sampling (balanced)    
            positions = max_the_max(y)
        else:
            raise ValueError("Type can be only  0, 1, 2 or 3")


        #print(cluster[0])
        # normalize each element of the cluster
        cluster = apply_translation(cluster[0])
        cluster = apply_randomrotations(cluster)
        cluster = cluster.double().to(device)

        # Salva l'ordine originale dei dati
        original_order = np.arange(len(cluster))
        #print(original_order)
        # Esegui lo shuffle dei dati
        np.random.shuffle(original_order)
        #print(original_order)
        cluster = cluster[original_order]

        # create an empty tensor that will collect the trasformed fragments
        cluster_transformed = torch.Tensor([]).to(device)

        sub_cluster_start = 0
        while sub_cluster_start < len(matrix[0][0]):
            sub_cluster_end = min(sub_cluster_start + 16, len(matrix[0][0]))
            cluster_subset = cluster[sub_cluster_start:sub_cluster_end]
            cluster_subset = cluster_subset.to(device)
            # apply the PCT
            point_clouds_transformed = self.ptc_net(cluster_subset)
            # append the results
            cluster_transformed = torch.cat((cluster_transformed, point_clouds_transformed), dim=0)
            sub_cluster_start += 16


        #print(cluster_transformed)
        sorted_indices = np.argsort(original_order)

        # Applica il riordino a cluster_transformed
        cluster_transformed = cluster_transformed[sorted_indices]
        # Ripristina l'ordine originale dei dati trasformati
        #cluster_transformed = cluster_transformed[torch.tensor(original_order)]
        #print(cluster_transformed)
        # given the positions, the matrix and the transformed cluster create the couples
        frags_a, frags_b, labels = CreateCouples_intra_train_masked_pos(cluster_transformed, matrix[0], positions)
        
        

        #######
        # Shuffle the pairs and labels randomly
        #perm = torch.randperm(frags_a.size(0))
        #frags_a = frags_a[perm]
        #frags_b = frags_b[perm]
        #labels = labels[perm]
        #######

        frags_a = frags_a.double().to(device)
        frags_b = frags_b.double().to(device)
        labels = labels.to(device)
        
        # Calculate positive and negative couples
        x_mult = frags_a * frags_b  # Sum the two elements of the couples
        x_sum = frags_a * frags_b   # Multiply the two elements of the couples

        # Combine positive and negative couples into a single tensor
        #positive_couples = torch.cat((x_mult[labels == 1], x_sum[labels == 1]), dim=1)
        #negative_couples = torch.cat((x_mult[labels == 0], x_sum[labels == 0]), dim=1)
        
        embeddings = torch.cat((x_mult, x_sum), dim = 1)

        cosine_similarities = F.cosine_similarity(frags_a, frags_b)

        min_value = torch.min(cosine_similarities)
        max_value = torch.max(cosine_similarities)
        scaled_similarities = (cosine_similarities - min_value) / (max_value - min_value)
        one_minus_scaled = 1 - scaled_similarities

        result_tensor = torch.cat((one_minus_scaled.view(-1, 1), scaled_similarities.view(-1, 1)), dim=1) 
        return result_tensor, labels, frags_a, frags_b, embeddings
    



class PairModel2(nn.Module):
    def __init__(self):
        super().__init__()
        
        output_channels = 2 # it's a binary classification

      
        
        self.relu = nn.ReLU()
            
        # classificator
        self.linear0 = nn.Linear(2048, 1024, bias=False)
        self.bn0 = nn.BatchNorm1d(1024)
        self.dp0 = nn.Dropout(p=0.5)
        self.linear1 = nn.Linear(1024, 512, bias=False)
        self.bn1 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=0.3)
        self.linear3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dp3 = nn.Dropout(p=0.3)
        self.linear4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dp4 = nn.Dropout(p=0.3)
        self.linear5 = nn.Linear(64, output_channels)
        
    def forward(self, x_1, x_2):
       
        x_mult = x_1 * x_2 # sum the two elements of the couples
        x_sum = x_1 * x_2 # multiply the two elements of the couples
        x = torch.cat((x_mult, x_sum), dim = 1) 

        # classificator
        if x_1.shape[0] > 1:
            x = self.relu(self.bn0(self.linear0(x)))
            #x = self.dp0(x)
            x = self.relu(self.bn1(self.linear1(x)))
            x = self.dp1(x)
            x = self.relu(self.bn2(self.linear2(x)))
            x = self.dp2(x)
            x = self.relu(self.bn3(self.linear3(x)))
            x = self.dp3(x)
            x = self.relu(self.bn4(self.linear4(x)))
            x = self.dp4(x)
            x = self.linear5(x)

        # If we have only one pair, applying batch normalization doesn't make sense.
        else: 
            x = self.relu(self.linear0(x))
            #x = self.dp0(x)
            x = self.relu(self.linear1(x))
            x = self.dp1(x)
            x = self.relu(self.linear2(x))
            x = self.dp2(x)
            x = self.relu(self.linear3(x))
            x = self.dp3(x)
            x = self.relu(self.linear4(x))
            x = self.dp4(x)
            x = self.linear5(x)
                    
        return x

In [ ]:
device=use_GPU()

NVIDIA GeForce RTX 4080 is available and being used


In [ ]:
train = train.tolist()
val = val.tolist()
# We process one cluster at a time.
train_loader = DataLoader(train, batch_size=1)
val_loader = DataLoader(val, batch_size=1)

In [ ]:
model = Model2_mod().to(device)
model.double()

Model2_mod(
  (ptc_net): Branch(
    (conv1): Conv1d(7, 64, kernel_size=(1,), stride=(1,), bias=False)
    (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (gather_local_0): Local_op(
      (conv1): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
      (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (gather_local_1): Local_op(
      (conv1): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
      (conv2): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=Tru

In [ ]:
W_stored = torch.load(r'C:\\Users\\Alessandro\\Desktop\\Tesi\\PairModel\\Check_points\\1010_180133_13+lr=0.0001+leggermenteunbalanced.pt')
model.load_state_dict(W_stored)


<All keys matched successfully>

In [ ]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, m=2.0):
        super(ContrastiveLoss, self).__init__()
        self.m = m

    def forward(self, y1, y2, d):
        euc_dist = torch.nn.functional.pairwise_distance(y1, y2)

        if d.dim() == 0:  # Se d è uno scalare
            if d == 0:
                return torch.mean(torch.pow(euc_dist, 2))  # Distanza quadratica
            else:  # d == 1
                delta = self.m - euc_dist
                delta = torch.clamp(delta, min=0.0, max=None)
                return torch.mean(torch.pow(delta, 2))
        else:  # Se d è un tensore di valori 0 e 1
            is_same = d == 0
            is_diff = d == 1

            loss_same = torch.pow(euc_dist[is_same], 2).mean() if torch.any(is_same) else torch.tensor(0.0).to(euc_dist.device)
            loss_diff = torch.pow(torch.clamp(self.m - euc_dist[is_diff], min=0.0), 2).mean() if torch.any(is_diff) else torch.tensor(0.0).to(euc_dist.device)

            return (loss_same + loss_diff) / (1.0 + torch.any(is_same).float() + torch.any(is_diff).float())

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="first_trial", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
  
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0004,
      "architecture": "Model2_mod",
      "dataset": "max_the_max sampled",
      "epochs": 5,
      "weight_decay": 0.00001,
      "W_crossentropy":1,
      "W_contrastive":0.05,
      "type_of_couples": 1,
      "seed": seed
      })
      
config = wandb.config


criterion = nn.CrossEntropyLoss()
contrast_criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
num_epochs = config.epochs
best_val_accuracy = 0.0 



#miner = miners.MultiSimilarityMiner()
#loss_func = losses.TripletMarginLoss()




checkpoint_dir = r'C:\\Users\\Alessandro\\Desktop\\Tesi\\PairModel\\Check_points'
results_dir = r'C:\\Users\\Alessandro\\Desktop\\Tesi\\PairModel\\results'

checkpoint_interval = 1  


for epoch in range(num_epochs):
    list_of_results=[]
    list_of_true=[]

    train_of_results=[]
    train_of_true=[]
    model.train() 


    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0


    ###########
    ## Train ##
    ###########
    

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for batch_data in progress_bar:

        optimizer.zero_grad() 

        matrix, cluster = batch_data

        cluster_outputs_, labels_, frags_a, frags_b, embeddings = model(matrix, cluster,config.type_of_couples)
        one_hot_labels_ = F.one_hot(labels_,2)
        #hard_pairs = miner(cluster_outputs_, labels_)

        train_of_true.append(labels_.tolist())

        #print(len(hard_pairs))
        #print(hard_pairs[0])
        #loss__ = loss_func(torch.cat((frags_a, frags_b), dim = 1), labels_)
        loss_ = criterion(cluster_outputs_, one_hot_labels_.float())
        contrast_loss = contrast_criterion(frags_a, frags_b, labels_)
       
        loss = loss_ + config.W_contrastive*contrast_loss
        loss.backward()
        optimizer.step()
       
        total_loss += loss.item()

        _, predicted = torch.max(cluster_outputs_.data, 1)

        train_of_results.append(predicted.tolist())

        total_samples += one_hot_labels_.size(0)
        correct_predictions += (predicted == labels_).sum().item()

        progress_bar.set_postfix({'Loss': loss.item(), 'Accuracy': correct_predictions / total_samples})

    accuracy = correct_predictions / total_samples
    train_loss = total_loss/len(train_loader)

    metrics_train = {"train_loss": train_loss, 
                       "accuracy": accuracy}
    wandb.log(metrics_train)

    ###############
    ## Inference ##
    ###############

    model.eval()  
    
    val_loss_ = 0.0
    v_contrast_loss= 0.0
    val_correct_predictions = 0
    val_total_samples = 0


    with torch.no_grad():
        for val_batch in val_loader:

            val_matrix, val_cluster = val_batch
            
            val_outputs_, val_labels_, v_frags_a, v_frags_b, embeddings = model(val_matrix, val_cluster, config.type_of_couples)
            
            list_of_true.append(val_labels_.tolist())

            one_hot_labels_val_ = F.one_hot(val_labels_,2)
             
   
            val_loss_ += criterion(val_outputs_, one_hot_labels_val_.float()).item()
            v_contrast_loss += contrast_criterion(v_frags_a, v_frags_b, val_labels_).item() 
           
            val_loss = val_loss_ + config.W_contrastive*v_contrast_loss

            _, val_predicted = torch.max(val_outputs_.data, 1)
            list_of_results.append(val_predicted.tolist())
            val_total_samples += one_hot_labels_val_.size(0)
            val_correct_predictions += (val_predicted == val_labels_).sum().item()

        val_accuracy = val_correct_predictions / val_total_samples
        val_loss /= len(val_loader)
    
    val_metrics = {"val_loss": val_loss, 
                       "val_accuracy": val_accuracy}
    wandb.log(val_metrics)                   

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Training Accuracy: {accuracy:.4f}, '
          f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    result_path = os.path.join(results_dir, f"+lr=0.0004results_epoch_{epoch+1}")              #############################
    os.makedirs(result_path, exist_ok=True)

    with open(os.path.join(result_path, "list_of_true.pkl"), 'wb') as f:
        pickle.dump(list_of_true, f)

    with open(os.path.join(result_path, "list_of_results.pkl"), 'wb') as f:
        pickle.dump(list_of_results, f)

    train_result_path = os.path.join(results_dir, f"TRAIN+lr=0.0004_results_epoch_{epoch+1}")             ##################  
    os.makedirs(train_result_path, exist_ok=True)

    with open(os.path.join(train_result_path, "train_of_true.pkl" ), 'wb') as f:
        pickle.dump(train_of_true, f)

    with open(os.path.join(train_result_path, "train_of_results.pkl"), 'wb') as f:
        pickle.dump(train_of_results, f)    

    current_time = datetime.datetime.now()
    checkpoint_name = f"{current_time.strftime('%m%d_%H%M%S')}_{epoch+1}+lr=0.0004+sampled_coupels.pt"    #################
    checkpoint_path = os.path.join(checkpoint_dir, checkpoint_name)
    torch.save(model.state_dict(), checkpoint_path)
    

KeyboardInterrupt: 

In [ ]:
test = test.tolist()

# We process one cluster at a time.
test_loader = DataLoader(test, batch_size=1)


In [ ]:
W_stored = torch.load(r'C:\\Users\\Alessandro\\Desktop\\Tesi\\PairModel\\Check_points\\1006_171026_3+2contrastivelosses.pt')

model.load_state_dict(W_stored)

<All keys matched successfully>

In [ ]:
model.eval() 
epoch=0 
list_of_results=[]
list_of_true=[]    
val_loss_ = 0.0
v_contrast_loss= 0.0
val_correct_predictions = 0
val_total_samples = 0
criterion = nn.CrossEntropyLoss()
contrast_criterion = ContrastiveLoss()
results_dir = r'C:\\Users\\Alessandro\\Desktop\\Tesi\\PairModel\\results'

with torch.no_grad():
        for val_batch in test_loader:

            val_matrix, val_cluster = val_batch
            
            val_outputs_, val_labels_, v_frags_a, v_frags_b, embeddings = model(val_matrix, val_cluster)
           
            list_of_true.append(val_labels_.tolist())

            one_hot_labels_val_ = F.one_hot(val_labels_,2)
             
            
            val_loss_ += criterion(val_outputs_, one_hot_labels_val_.float()).item()
            v_contrast_loss += contrast_criterion(v_frags_a, v_frags_b, val_labels_).item() 
           
            val_loss = val_loss_ + 0.05*v_contrast_loss

            _, val_predicted = torch.max(val_outputs_.data, 1)
            list_of_results.append(val_predicted.tolist())
            val_total_samples += one_hot_labels_val_.size(0)
            val_correct_predictions += (val_predicted == val_labels_).sum().item()

val_accuracy = val_correct_predictions / val_total_samples
val_loss /= len(val_loader)
print(val_accuracy)
wandb.summary['test_accuracy'] = val_accuracy
wandb.finish()

0.6339391045446885


accuracy,▁▇▇█▇
train_loss,█▁▁▁▁
val_accuracy,▁▃▃█▁
val_loss,▅█▁▄▇
accuracy,0.62644
test_accuracy,0.63394
train_loss,0.69466
val_accuracy,0.63237
val_loss,0.69288
